In [1]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

Matplotlib is building the font cache; this may take a moment.


In [ ]:
## Data
## Cleaned SBFE: Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Data Modeling Clean/sbfe_seleid_append_deduped_cleaned_0501.parquet
## Scout data read in: Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/SCOUT transacations/

In [2]:
path = "Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Data Modeling Clean/sbfe_seleid_append_deduped_cleaned_0501.parquet"
sbfe = pd.read_adls(path, reader = pd.read_parquet)
print(sbfe.shape)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EFZYCUHFK to authenticate.
(79429086, 20)


In [4]:
sbfe["count"] = 1
fmt = make_format(cuts = [-np.inf, -1, 0, 1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 100, np.inf])
temp_roll = sbfe.groupby(by = ['seleid'])['count'].sum().reset_index(name='count')
print(temp_roll["count"].describe())
freq(temp_roll["count"], format = [fmt], observed = True)

count    1.094990e+07
mean     7.253866e+00
std      8.144695e+00
min      1.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      1.000000e+01
max      3.310000e+02
Name: count, dtype: float64


,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,3373468.0,0.308082,3373468.0,0.308082
2,1804139.0,0.164763,5177607.0,0.472845
3,683840.0,0.062452,5861447.0,0.535297
4,323255.0,0.029521,6184702.0,0.564818
5,215001.0,0.019635,6399703.0,0.584453
6-10,2061596.0,0.188275,8461299.0,0.772729
11-15,730852.0,0.066745,9192151.0,0.839474
16-20,198579.0,0.018135,9390730.0,0.857609
21-30,1455583.0,0.132931,10846313.0,0.990540


In [5]:
fmt_1 = make_format(cuts = [-np.inf, 0, np.inf])
fmt_2 = make_format(cuts = [-np.inf, 0, 43, np.inf])
freq(sbfe.seleid.astype(float), sbfe.overallweight.astype(float), format = [fmt_1, fmt_2], cross=False, observed = True)

Count       Pct  Cuml Count  Cuml Pct
seleid overallweight                                            
1+     1-43                961.0  0.000012       961.0  0.000012
       44+            79428125.0  0.999988  79429086.0  1.000000

In [3]:
sbfe.columns

Index(['accountnumber_x', 'companyname', 'alternatecompanyname_x', 'addr',
       'city', 'state', 'zip', 'businessphone', 'taxidnumber', 'historydate_x',
       'sufficient_input', 'accountnumber_y', 'alternatecompanyname_y',
       'historydate_y', 'powid', 'proxid', 'seleid', 'orgid', 'ultid',
       'overallweight'],
      dtype='object')

In [4]:
wb = TableWriter(filename = "./_temp/state.csv", options={'nan_inf_to_errors': True}, overwrite = True)
wb.write_table(
    freq(sbfe.state),
    sheetname="biv",
    conditional_fmt_cols=[1],
)
wb.close()

In [6]:
northeast = ['NH', 'VT' , 'MA' , 'RI' , 'CT' , 'NY' , 'NJ' , 'PA' , 'ME' ]
midwest = ['MI' , 'IN' , 'WI' , 'IL' , 'MN' , 'IA' , 'MO' , 'ND' , 'SD' , 'NE' , 'KS', 'OH' ]
south = ['MD' , 'VA' , 'WV' , 'KY' , 'NC' , 'SC' , 'TN' , 'GA' , 'FL' , 'AL' , 'MS' , 'AR' , 'LA' , 'TX' , 'OK' , 'DE' ]
west = ['ID' , 'WY' , 'CO' , 'NM' , 'AZ' , 'UT' , 'NV' , 'CA' , 'OR' , 'WA' , 'AK' , 'HI' , 'MT' ]


sbfe["region_flag"] = np.where(sbfe["state"].str.upper().isin(northeast), "Northeast", 
                                     np.where(sbfe["state"].str.upper().isin(midwest), "Midwest", 
                                             np.where(sbfe["state"].str.upper().isin(south), "South", 
                                                     np.where(sbfe["state"].str.upper().isin(west), "West", "_Other"))))
freq(sbfe.region_flag)

,Count,Pct,Cuml Count,Cuml Pct
region_flag,,,,
Midwest,12737138.0,0.160359,12737138.0,0.160359
Northeast,15641397.0,0.196923,28378535.0,0.357281
South,28547220.0,0.359405,56925755.0,0.716687
West,20584913.0,0.259161,77510668.0,0.975847
_Other,1918418.0,0.024153,79429086.0,1.000000


In [9]:
path = r"./_temp/SDI_ACS2015_zctaallvars (1).xlsx"
sdi = pd.read_excel(path, sheet_name = "ACS2015_zctaallvars", dtype = str)
sdi.shape

# ZCTA is the zip code and SDI_Score is the SDI.  
# There may be a very small amount of zip’s that don’t have an SDI due to not having the data available for that, but it should be a very small amount.
# Match to zip5: you should do that and front load with a 0 for the 4 digit values
sdi = sdi[["zcta", "sdi_score"]]
sdi['zcta_padded'] = sdi['zcta'].apply(lambda x: x.zfill(5))
sdi['sdi_score'] = sdi['sdi_score'].astype(float)
sbfe = sbfe.merge(sdi, how = "left", left_on = "zip", right_on = "zcta_padded")
print(sbfe.shape)
print(sbfe["sdi_score"].describe())

# if [Sdi Score] > 80 then 'Depressed'ELSEIF [Sdi Score] > 20 THEN 'Average'
# ELSEIF [Sdi Score] >=0 THEN 'Affluent'
# ELSE 'Other'
# The higher the score the more depressed socioeconomic level.  
# Also, each value represents roughly 1% of the population of the US, which worked pretty good using our Prospect Generator database
sbfe["sdi_flag"] = np.where(sbfe["sdi_score"] >80, "Depressed", 
                                  np.where(sbfe["sdi_score"] > 20, "Average", 
                                          np.where(sbfe["sdi_score"] >=0, "Affluent", "_Other")))
freq(sbfe.sdi_flag)

(79429086, 24)
count    6.020502e+07
mean     4.228267e+01
std      2.847645e+01
min      1.000000e+00
25%      1.700000e+01
50%      3.800000e+01
75%      6.600000e+01
max      1.000000e+02
Name: sdi_score, dtype: float64


,Count,Pct,Cuml Count,Cuml Pct
sdi_flag,,,,
Affluent,17604937.0,0.221643,17604937.0,0.221643
Average,34601715.0,0.435630,52206652.0,0.657274
Depressed,7998371.0,0.100698,60205023.0,0.757972
_Other,19224063.0,0.242028,79429086.0,1.000000


Bad pipe message: %s [b'\x9d\xc5\x05\x96M[#\xcfSR\xcd\xbe\xb0Q\xb8\xebA\x05 \x90\x18\r\xc9*b5A\x1e\xee\x00\xb3=', b'\x8e\xe1w\xa1?\x00\xc0)\x07nC,,;\xa3`\x96\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00', b'\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00']
Bad pipe message: %s [b'\x1d\x00\x17\x00\x1e\x00\x19\x00\x18']
Bad pipe message: %s [b"J\xac\xec\xff\x91j\xf4\x9b\x028\x9as\xa9\xb8'e\x99Z\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0", b'3\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0']
Bad pipe message: %s [b'=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\

In [6]:
print(sbfe[sbfe["companyname"] == ""].shape)
print(sbfe[sbfe["alternatecompanyname_x"] == ""].shape)
print(sbfe[sbfe["addr"] == ""].shape)
print(sbfe[sbfe["city"] == ""].shape)
print(sbfe[sbfe["state"] == ""].shape)
print(sbfe[sbfe["zip"] == ""].shape)
print(sbfe[sbfe["taxidnumber"] == ""].shape)
print(sbfe[sbfe["businessphone"] == ""].shape)
print(sbfe[sbfe["historydate_x"] == ""].shape)

(0, 20)
(79429086, 20)
(0, 20)
(1624707, 20)
(1623153, 20)
(3790325, 20)
(76230542, 20)
(21962890, 20)
(0, 20)


Bad pipe message: %s [b"9\x14J\x17\x835@\x90\r\x0cj\x1a'`\xbd3\xaf` g\xb2\x0eAT\xbe\xc7m\x18\x83\xc9\x845\x17{a\xac\x9e~<\xe6\x03\x81Z\xee\xab\xe3\xf7\xf2\x01<%\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07"]
Bad pipe message: %s [b'\x08\t\x08\n\x08\x0b\x08']
Bad pipe message: %s [b"\xea\xd9\xa1>A\xacj\\\xd3\x9f\x05\xd1\xb8\xfc/g\x03\xdc\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\

In [5]:
sbfe.columns

Index(['accountnumber_x', 'companyname', 'alternatecompanyname_x', 'addr',
       'city', 'state', 'zip', 'businessphone', 'taxidnumber', 'historydate_x',
       'sufficient_input', 'accountnumber_y', 'alternatecompanyname_y',
       'historydate_y', 'powid', 'proxid', 'seleid', 'orgid', 'ultid',
       'overallweight'],
      dtype='object')

In [6]:
21962890 + 57466196

79429086

In [7]:
sbfe["full_addr"] = sbfe["addr"] + np.where(sbfe["city"] != "", ", " + sbfe["city"], "") + \
    np.where(sbfe["state"] != "", ", " + sbfe["state"], "") + np.where(sbfe["zip"] != "", ", " + sbfe["zip"].str.slice(0, 5), "")
sbfe["full_addr"]

0             115 GOLF COURSE RD STE E, LOGAN, UT, 84321
1                 3200 PALM TREE DR, LITHONIA, GA, 30038
2               1476 HIGHWAY 159 E, BELLVILLE, TX, 77418
5                492 KOLLER ST, SAN FRANCISCO, CA, 94110
6                107 LASSITER LANE, BELLVILLE, TX, 77418
                                ...                     
92855292       2022 BRIARCLIFFE BLVD, WHEATON, IL, 60189
92855294          4860 KINGS CT, RICHTON PARK, IL, 60471
92855295    1511 HOLCOMBS POND CT, ALPHARETTA, GA, 30022
92855296          1735 LONGVIEW WAY NW, SALEM, OR, 97304
92855297          13145 YERBA ST, N HOLLYWOOD, CA, 91605
Name: full_addr, Length: 79429086, dtype: object

In [10]:
sbfe["full_addr_clean"] = sbfe['full_addr'].str.replace('\W\s\,', '')
sbfe.loc[sbfe["full_addr_clean"]!=sbfe["full_addr"], ["full_addr_clean", "full_addr"]].head()

/tmp/ipykernel_6446/3677427696.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  sbfe["full_addr_clean"] = sbfe['full_addr'].str.replace('\W\s\,', '')


,full_addr_clean,full_addr
2156707,"11175 LAKEVIEW AV 1963, LUCERNE VALLEY, CA, 9...","11175 LAKEVIEW AV. , 1963, LUCERNE VALLEY, CA..."
4907592,"500 MS-12 MS 39759, STARKVILLE, MS, 39759","500 MS-12, , MS 39759, STARKVILLE, MS, 39759"
4935389,"500 MS-12 MS 39759, STARKVILLE, MS, 39759","500 MS-12, , MS 39759, STARKVILLE, MS, 39759"
7966573,"6521 SALTSBURG RD PA 15235, PITTSBURGH, PA, 15235","6521 SALTSBURG RD, , PA 15235, PITTSBURGH, PA,..."
15990890,"11175 LAKEVIEW AV 1963, LUCERNE VALLEY, CA, 9...","11175 LAKEVIEW AV. , 1963, LUCERNE VALLEY, CA..."


In [11]:
# temp = sbfe.head().copy()
# from geopy.geocoders import GoogleV3
# geolocator = GoogleV3(api_key='Your Google Maps API Key')
# def extract_clean_address(address):
#     try:
#         location = geolocator.geocode(address)
#         return location.address
#     except:
#         return ''
# temp['clean_full_addr'] = temp.apply(lambda x: extract_clean_address(x['full_addr']) , axis =1)

In [12]:
# temp[["clean_full_addr", "full_addr"]]

,clean_full_addr,full_addr
0,,"115 GOLF COURSE RD STE E, LOGAN, UT, 84321"
1,,"3200 PALM TREE DR, LITHONIA, GA, 30038"
2,,"1476 HIGHWAY 159 E, BELLVILLE, TX, 77418"
5,,"492 KOLLER ST, SAN FRANCISCO, CA, 94110"
6,,"107 LASSITER LANE, BELLVILLE, TX, 77418"


In [11]:
temp = sbfe[sbfe["full_addr_clean"]!= ""].copy()
print(temp.shape)
temp["count"] = 1
temp_roll = temp.groupby(by = ['full_addr_clean'])['count'].sum().reset_index(name='count')
temp_roll["count"].describe()

(79429086, 22)


count    1.283377e+07
mean     6.189067e+00
std      8.809741e+00
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      9.000000e+00
max      2.494000e+03
Name: count, dtype: float64

In [12]:
fmt = make_format(cuts = [-np.inf, -1, 0, 1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 100, np.inf])
freq(temp_roll["count"], format = [fmt], observed = True)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,4981561.0,0.388160,4981561.0,0.388160
2,2373355.0,0.184930,7354916.0,0.573091
3,583125.0,0.045437,7938041.0,0.618527
4,371687.0,0.028962,8309728.0,0.647489
5,214178.0,0.016689,8523906.0,0.664178
6-10,2000865.0,0.155906,10524771.0,0.820084
11-15,672554.0,0.052405,11197325.0,0.872489
16-20,235168.0,0.018324,11432493.0,0.890813
21-30,1217420.0,0.094861,12649913.0,0.985674


In [7]:
temp = sbfe[sbfe["taxidnumber"]!= ""].copy()
temp["count"] = 1
temp_roll = temp.groupby(by = ['taxidnumber'])['count'].sum().reset_index(name='count')
temp_roll["count"].describe()

count    1.897106e+06
mean     1.686012e+00
std      1.772816e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      8.260000e+02
Name: count, dtype: float64

In [24]:
fmt = make_format(cuts = [-np.inf, -1, 0, 1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 100, np.inf])
freq(temp_roll["count"], format = [fmt], observed = True)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,1261130.0,0.664765,1261130.0,0.664765
2,379205.0,0.199886,1640335.0,0.864651
3,89044.0,0.046937,1729379.0,0.911588
4,68373.0,0.036041,1797752.0,0.947629
5,56479.0,0.029771,1854231.0,0.977400
6-10,38744.0,0.020423,1892975.0,0.997822
11-15,2979.0,0.001570,1895954.0,0.999393
16-20,707.0,0.000373,1896661.0,0.999765
21-30,329.0,0.000173,1896990.0,0.999939


In [25]:
temp_roll = temp.groupby(by = ['seleid'])['count'].sum().reset_index(name='count')
print(temp_roll["count"].describe())
freq(temp_roll["count"], format = [fmt], observed = True)

count    1.959711e+06
mean     1.632151e+00
std      1.396700e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      1.410000e+02
Name: count, dtype: float64


,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,1357727.0,6.928200e-01,1357727.0,0.692820
2,358714.0,1.830443e-01,1716441.0,0.875864
3,83164.0,4.243687e-02,1799605.0,0.918301
4,62813.0,3.205218e-02,1862418.0,0.950353
5,52215.0,2.664423e-02,1914633.0,0.976998
6-10,41350.0,2.110005e-02,1955983.0,0.998098
11-15,2846.0,1.452255e-03,1958829.0,0.999550
16-20,585.0,2.985134e-04,1959414.0,0.999848
21-30,266.0,1.357343e-04,1959680.0,0.999984


In [17]:
temp.businessphone.is_unique

False

In [19]:
temp = sbfe[sbfe["businessphone"]!= ""].copy()
temp["count"] = 1
temp_roll = temp.groupby(by = ['businessphone'])['count'].sum().reset_index(name='count')
temp_roll["count"].describe()

count    6.213632e+06
mean     9.248407e+00
std      1.018456e+01
min      1.000000e+00
25%      2.000000e+00
50%      9.000000e+00
75%      1.200000e+01
max      5.094000e+03
Name: count, dtype: float64

In [20]:
fmt = make_format(cuts = [-np.inf, -1, 0, 1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 100, np.inf])
freq(temp_roll["count"], format = [fmt], observed = True)

,Count,Pct,Cuml Count,Cuml Pct
count,,,,
1,1419907.0,0.228515,1419907.0,0.228515
2,578543.0,0.093109,1998450.0,0.321623
3,276124.0,0.044438,2274574.0,0.366062
4,177727.0,0.028603,2452301.0,0.394665
5,113126.0,0.018206,2565427.0,0.412871
6-10,1771722.0,0.285135,4337149.0,0.698005
11-15,568128.0,0.091433,4905277.0,0.789438
16-20,183593.0,0.029547,5088870.0,0.818985
21-30,1033069.0,0.166258,6121939.0,0.985243


In [3]:
temp_roll = temp.groupby(by = ['seleid'])['count'].sum().reset_index(name='count')
print(temp_roll.shape)
print(temp_roll["count"].describe())
freq(temp_roll["count"], format = [fmt], observed = True)

NameError: name 'temp' is not defined